## Single Model Code
This script focuses on training and evaluating individual models (e.g., VGG16, ResNet18, GoogLeNet) on datasets like MNIST and CIFAR10, using a two-phase approach: feature extraction and fine-tuning.

In [ ]:
pip install torchviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
pip install opencv-python

In [ ]:
pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 108.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np
import time
import os
import copy
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration and Hyperparameters
NUM_CLASSES = 10
DATASET_LIST = ["MNIST", "CIFAR10"]
DATA_DIR = './data'
MODEL_LIST = ["vgg16", "resnet18", "googlenet"]
INPUT_SIZE = 224
BATCH_SIZE = 32
EPOCHS_FEATURE_EXTRACT = 5
EPOCHS_FINE_TUNE = 10
LR_CLASSIFIER = 1e-3
LR_FINE_TUNE_BASE = 1e-5
OPTIMIZER_TYPE = "Adam"
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs('./plots', exist_ok=True)

# Data Handling Functions
def get_transforms(dataset_name, input_size, is_train=True):
    transform_list = [transforms.Resize((input_size, input_size))]
    if dataset_name == "MNIST":
        transform_list.append(transforms.Grayscale(num_output_channels=3))
    if is_train and dataset_name == "CIFAR10":
        transform_list.extend([transforms.RandomHorizontalFlip(), transforms.RandomCrop(input_size, padding=4)])
    transform_list.extend([transforms.ToTensor(), transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)])
    return transforms.Compose(transform_list)

def get_dataloaders(dataset_name, batch_size, input_size, data_dir=DATA_DIR):
    train_transform = get_transforms(dataset_name, input_size, is_train=True)
    val_transform = get_transforms(dataset_name, input_size, is_train=False)
    if dataset_name == "MNIST":
        train_dataset = datasets.MNIST(root=data_dir, train=True, download=True, transform=train_transform)
        test_dataset = datasets.MNIST(root=data_dir, train=False, download=True, transform=val_transform)
        class_names = [str(i) for i in range(10)]
    elif dataset_name == "CIFAR10":
        train_dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=train_transform)
        test_dataset = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=val_transform)
        class_names = test_dataset.classes
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    dataloaders = {'train': train_loader, 'val': test_loader}
    dataset_sizes = {'train': len(train_dataset), 'val': len(test_dataset)}
    print(f"Dataset: {dataset_name}\n  Training samples: {dataset_sizes['train']}\n  Validation/Test samples: {dataset_sizes['val']}\n  Number of classes: {len(class_names)}")
    return dataloaders, dataset_sizes, class_names

# Model Adaptation Functions
def get_model(model_name, num_classes, pretrained=True):
    print(f"Loading model: {model_name} (pretrained={pretrained})")
    if model_name == "vgg16":
        model = models.vgg16(pretrained=pretrained)
        input_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(input_features, num_classes)
    elif model_name == "resnet18":
        model = models.resnet18(pretrained=pretrained)
        input_features = model.fc.in_features
        model.fc = nn.Linear(input_features, num_classes)
    elif model_name == "googlenet":
        model = models.googlenet(pretrained=pretrained, aux_logits=True)
        input_features = model.fc.in_features
        model.fc = nn.Linear(input_features, num_classes)
        if model.aux1 is not None:
            input_features_aux1 = model.aux1.fc2.in_features
            model.aux1.fc2 = nn.Linear(input_features_aux1, num_classes)
        if model.aux2 is not None:
            input_features_aux2 = model.aux2.fc2.in_features
            model.aux2.fc2 = nn.Linear(input_features_aux2, num_classes)
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    print(f"  Replaced final layer. Original input features: {input_features}, New output classes: {num_classes}")
    return model

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        print("Freezing base model parameters for feature extraction.")
        for param in model.parameters():
            param.requires_grad = False
    else:
        print("Unfreezing all model parameters for fine-tuning.")
        for param in model.parameters():
            param.requires_grad = True

# Training Function
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, device, num_epochs=10):
    start_time = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}\n{"-" * 10}')
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    if hasattr(outputs, 'logits'):  # For GoogLeNet
                        loss = criterion(outputs.logits, labels)
                        if phase == 'train':
                            loss += 0.3 * criterion(outputs.aux_logits2, labels)
                            loss += 0.3 * criterion(outputs.aux_logits1, labels)
                        _, preds = torch.max(outputs.logits, 1)
                    else:
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f'  New best validation accuracy: {best_acc:.4f}')
        print()
    time_elapsed = time.time() - start_time
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    model.load_state_dict(best_model_wts)
    return model, history

# Evaluation Function
def evaluate_model(model, dataloader, device, class_names):
    model.eval()
    all_preds, all_labels = [], []
    print("\nEvaluating on test set...")
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            if hasattr(outputs, 'logits'):  # For GoogLeNet
                outputs = outputs.logits
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    class_report = classification_report(all_labels, all_preds, target_names=class_names, zero_division=0, output_dict=True)
    print("Evaluation complete.")
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': conf_matrix,
        'classification_report': class_report
    }

# Visualization Functions
def plot_dataset_samples(dataloader, class_names, dataset_name):
    images, labels = next(iter(dataloader))
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            img = images[i].permute(1, 2, 0).numpy()
            img = img * np.array(IMAGENET_STD) + np.array(IMAGENET_MEAN)  # Denormalize
            img = np.clip(img, 0, 1)
            ax.imshow(img)
            ax.set_title(class_names[labels[i]])
            ax.axis('off')
    plt.suptitle(f'Sample Images from {dataset_name}')
    plt.tight_layout()
    plt.savefig(f'./plots/dataset_samples_{dataset_name.lower()}.png')
    plt.close()

def plot_combined_history(history_fe, history_ft, dataset_name, model_name):
    train_loss = history_fe['train_loss'] + history_ft['train_loss']
    val_loss = history_fe['val_loss'] + history_ft['val_loss']
    train_acc = history_fe['train_acc'] + history_ft['train_acc']
    val_acc = history_fe['val_acc'] + history_ft['val_acc']
    epochs = range(1, len(train_loss) + 1)

    # Accuracy plot
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
    plt.axvline(x=EPOCHS_FEATURE_EXTRACT, color='k', linestyle='--', label='Feature Extraction End')
    plt.title(f'Training and Validation Accuracy for {model_name} on {dataset_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(f'./plots/{dataset_name.lower()}_accuracy_curves_{model_name}.png')
    plt.close()

    # Loss plot
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.axvline(x=EPOCHS_FEATURE_EXTRACT, color='k', linestyle='--', label='Feature Extraction End')
    plt.title(f'Training and Validation Loss for {model_name} on {dataset_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'./plots/{dataset_name.lower()}_loss_curves_{model_name}.png')
    plt.close()

def show_sample_predictions(model, dataloader, class_names, device, experiment_name):
    model.eval()
    images, labels = next(iter(dataloader))
    images, labels = images.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(images)
        if hasattr(outputs, 'logits'):  # For GoogLeNet
            outputs = outputs.logits
        _, preds = torch.max(outputs, 1)
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    for i, ax in enumerate(axes.flat):
        img = images[i].cpu().permute(1, 2, 0).numpy()
        img = img * np.array(IMAGENET_STD) + np.array(IMAGENET_MEAN)  # Denormalize
        img = np.clip(img, 0, 1)
        ax.imshow(img)
        true_label = class_names[labels[i]]
        pred_label = class_names[preds[i]]
        ax.set_title(f"True: {true_label}\nPred: {pred_label}", color='green' if true_label == pred_label else 'red')
        ax.axis('off')
    plt.tight_layout()
    plt.savefig(f'./plots/{experiment_name}_sample_predictions.png')
    plt.close()

def plot_confusion_matrix(conf_matrix, class_names, experiment_name):
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix for {experiment_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(f'./plots/confusion_matrix_{experiment_name}.png')
    plt.close()

def plot_model_comparison(results, dataset_name):
    models = list(results.keys())
    accuracies = [results[model]['accuracy'] for model in models]
    plt.figure(figsize=(8, 6))
    plt.bar(models, accuracies, color='skyblue')
    plt.title(f'Model Performance Comparison on {dataset_name}')
    plt.xlabel('Model')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    for i, v in enumerate(accuracies):
        plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
    plt.savefig(f'./plots/model_comparison_{dataset_name.lower()}.png')
    plt.close()

# Main Execution Block
if __name__ == "__main__":
    for dataset_name in DATASET_LIST:
        dataloaders, dataset_sizes, class_names = get_dataloaders(dataset_name, BATCH_SIZE, INPUT_SIZE)
        plot_dataset_samples(dataloaders['train'], class_names, dataset_name)
        results = {}
        for model_name in MODEL_LIST:
            experiment_name = f"{model_name}_{dataset_name}"
            print(f"\n{'='*20} Starting Experiment: {experiment_name} {'='*20}")
            model = get_model(model_name, NUM_CLASSES, pretrained=True).to(device)
            criterion = nn.CrossEntropyLoss()

            # Feature Extraction Phase
            print("\n--- Phase 1: Feature Extraction ---")
            set_parameter_requires_grad(model, feature_extracting=True)
            if model_name == "vgg16":
                for param in model.classifier[6].parameters():
                    param.requires_grad = True
                params_to_update_fe = list(model.classifier[6].parameters())
            elif model_name == "resnet18":
                for param in model.fc.parameters():
                    param.requires_grad = True
                params_to_update_fe = list(model.fc.parameters())
            elif model_name == "googlenet":
                for param in model.fc.parameters():
                    param.requires_grad = True
                if model.aux1 is not None:
                    for param in model.aux1.fc2.parameters():
                        param.requires_grad = True
                if model.aux2 is not None:
                    for param in model.aux2.fc2.parameters():
                        param.requires_grad = True
                params_to_update_fe = list(model.fc.parameters())
                if model.aux1 is not None:
                    params_to_update_fe.extend(model.aux1.fc2.parameters())
                if model.aux2 is not None:
                    params_to_update_fe.extend(model.aux2.fc2.parameters())
            optimizer_fe = optim.Adam(params_to_update_fe, lr=LR_CLASSIFIER) if OPTIMIZER_TYPE == "Adam" else optim.SGD(params_to_update_fe, lr=LR_CLASSIFIER)
            model_fe, history_fe = train_model(model, dataloaders, dataset_sizes, criterion, optimizer_fe, device, EPOCHS_FEATURE_EXTRACT)

            # Fine-Tuning Phase
            print("\n--- Phase 2: Fine-Tuning ---")
            set_parameter_requires_grad(model, feature_extracting=False)
            if model_name == "vgg16":
                base_params = list(model.features.parameters())
                classifier_params = list(model.classifier[6].parameters())
            elif model_name == "resnet18":
                base_params = [p for p in model.parameters() if p not in set(model.fc.parameters())]
                classifier_params = list(model.fc.parameters())
            elif model_name == "googlenet":
                classifier_params = list(model.fc.parameters())
                if model.aux1 is not None:
                    classifier_params.extend(model.aux1.fc2.parameters())
                if model.aux2 is not None:
                    classifier_params.extend(model.aux2.fc2.parameters())
                base_params = [p for p in model.parameters() if p not in set(classifier_params)]
            params_to_update_ft = [{'params': base_params, 'lr': LR_FINE_TUNE_BASE}, {'params': classifier_params, 'lr': LR_CLASSIFIER}]
            optimizer_ft = optim.Adam(params_to_update_ft) if OPTIMIZER_TYPE == "Adam" else optim.SGD(params_to_update_ft)
            model_ft, history_ft = train_model(model, dataloaders, dataset_sizes, criterion, optimizer_ft, device, EPOCHS_FINE_TUNE)

            # Evaluation
            final_metrics = evaluate_model(model_ft, dataloaders['val'], device, class_names)
            results[model_name] = final_metrics
            print(f"\n--- Results for {experiment_name} ---\nFinal Test Set Metrics:\n  Accuracy: {final_metrics['accuracy']:.4f}")

            # Visualizations
            plot_combined_history(history_fe, history_ft, dataset_name, model_name)
            show_sample_predictions(model_ft, dataloaders['val'], class_names, device, experiment_name)
            if model_name == "resnet18" and dataset_name == "CIFAR10":
                plot_confusion_matrix(final_metrics['confusion_matrix'], class_names, experiment_name)

        # Model Comparison
        plot_model_comparison(results, dataset_name)

100%|██████████| 9.91M/9.91M [00:00<00:00, 42.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.16MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 12.9MB/s]

Dataset: MNIST
  Training samples: 60000
  Validation/Test samples: 10000
  Number of classes: 10



==================== Starting Experiment: vgg16_MNIST ====================
Loading model: vgg16 (pretrained=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 230MB/s]


  Replaced final layer. Original input features: 4096, New output classes: 10

--- Phase 1: Feature Extraction ---
Freezing base model parameters for feature extraction.
Epoch 1/5
----------
Train Loss: 0.5232 Acc: 0.8288
Val Loss: 0.2312 Acc: 0.9348
  New best validation accuracy: 0.9348

Epoch 2/5
----------
Train Loss: 0.4267 Acc: 0.8614
Val Loss: 0.2012 Acc: 0.9413
  New best validation accuracy: 0.9413

Epoch 3/5
----------
Train Loss: 0.4181 Acc: 0.8623
Val Loss: 0.1963 Acc: 0.9395

Epoch 4/5
----------
Train Loss: 0.4198 Acc: 0.8670
Val Loss: 0.1856 Acc: 0.9476
  New best validation accuracy: 0.9476

Epoch 5/5
----------
Train Loss: 0.4231 Acc: 0.8677
Val Loss: 0.1849 Acc: 0.9455

Training complete in 4m 60s
Best val Acc: 0.947600

--- Phase 2: Fine-Tuning ---
Unfreezing all model parameters for fine-tuning.
Epoch 1/10
----------
Train Loss: 0.1007 Acc: 0.9692
Val Loss: 0.0366 Acc: 0.9873
  New best validation accuracy: 0.9873

Epoch 2/10
----------
Train Loss: 0.0354 Acc: 0.989

...

  Replaced final layer. Original input features: 4096, New output classes: 10

--- Phase 1: Feature Extraction ---
Freezing base model parameters for feature extraction.
Epoch 1/5
----------
Train Loss: 0.7287 Acc: 0.7490
Val Loss: 0.6120 Acc: 0.7882
  New best validation accuracy: 0.7882

Epoch 2/5
----------
Train Loss: 0.7091 Acc: 0.7670
Val Loss: 0.5501 Acc: 0.8134
  New best validation accuracy: 0.8134

Epoch 3/5
----------
Train Loss: 0.7203 Acc: 0.7684
Val Loss: 0.5197 Acc: 0.8232
  New best validation accuracy: 0.8232

Epoch 4/5
----------
Train Loss: 0.7237 Acc: 0.7709
Val Loss: 0.5202 Acc: 0.8255
  New best validation accuracy: 0.8255

Epoch 5/5
----------
Train Loss: 0.7320 Acc: 0.7702
Val Loss: 0.5414 Acc: 0.8184

Training complete in 4m 39s
Best val Acc: 0.825500

--- Phase 2: Fine-Tuning ---
Unfreezing all model parameters for fine-tuning.
Epoch 1/10
----------
Train Loss: 0.4249 Acc: 0.8557
Val Loss: 0.3148 Acc: 0.8901
  New best validation accuracy: 0.8901

Epoch 2/10
-

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  Replaced final layer. Original input features: 512, New output classes: 10

--- Phase 1: Feature Extraction ---
Freezing base model parameters for feature extraction.
Epoch 1/5
----------
Train Loss: 0.7911 Acc: 0.7412
Val Loss: 0.6185 Acc: 0.7887
  New best validation accuracy: 0.7887

Epoch 2/5
----------
Train Loss: 0.6294 Acc: 0.7832
Val Loss: 0.5985 Acc: 0.7962
  New best validation accuracy: 0.7962

Epoch 3/5
----------
Train Loss: 0.6107 Acc: 0.7890
Val Loss: 0.6331 Acc: 0.7833

Epoch 4/5
----------
Train Loss: 0.6036 Acc: 0.7921
Val Loss: 0.6009 Acc: 0.7958

Epoch 5/5
----------
Train Loss: 0.5934 Acc: 0.7963
Val Loss: 0.5708 Acc: 0.8079
  New best validation accuracy: 0.8079

Training complete in 4m 39s
Best val Acc: 0.807900

--- Phase 2: Fine-Tuning ---
Unfreezing all model parameters for fine-tuning.
Epoch 1/10
----------
Train Loss: 0.3810 Acc: 0.8689
Val Loss: 0.2498 Acc: 0.9150
  New best validation accuracy: 0.9150

Epoch 2/10
----------
Train Loss: 0.2371 Acc: 0.9191

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/googlenet.py:341: UserWarning: auxiliary heads in the pretrained googlenet model are NOT pretrained, so make sure to train them
  warnings.warn(


Epoch 1/5
----------
Train Loss: 1.8057 Acc: 0.7218
Val Loss: 0.6345 Acc: 0.7846
  New best validation accuracy: 0.7846

Epoch 2/5
----------
Train Loss: 1.5712 Acc: 0.7607
Val Loss: 0.5966 Acc: 0.7962
  New best validation accuracy: 0.7962

Epoch 3/5
----------
Train Loss: 1.5477 Acc: 0.7652
Val Loss: 0.5683 Acc: 0.8045
  New best validation accuracy: 0.8045

Epoch 4/5
----------
Train Loss: 1.5507 Acc: 0.7648
Val Loss: 0.5698 Acc: 0.8021

Epoch 5/5
----------
Train Loss: 1.5403 Acc: 0.7693
Val Loss: 0.5748 Acc: 0.8048
  New best validation accuracy: 0.8048

Training complete in 4m 39s
Best val Acc: 0.804800

--- Phase 2: Fine-Tuning ---
Unfreezing all model parameters for fine-tuning.
Epoch 1/10
----------
Train Loss: 1.1462 Acc: 0.8528
Val Loss: 0.2755 Acc: 0.9044
  New best validation accuracy: 0.9044

Epoch 2/10
----------
Train Loss: 0.8425 Acc: 0.9004
Val Loss: 0.2168 Acc: 0.9265
  New best validation accuracy: 0.9265

Epoch 3/10
----------
Train Loss: 0.7083 Acc: 0.9207
Val Los

## Parallel Model Code
This script is dedicated to training and evaluating parallel architectures, where two models are combined to extract features, which are then concatenated and passed through a final classifier.

1. Imports and Device Configuration

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


2. Data Transformations

In [21]:
# Data transformations
transform_mnist = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels for pretrained models
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

transform_cifar10 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

3. Dataset Loading

In [22]:
# Load datasets
print("Loading datasets...")

# MNIST dataset
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)

# CIFAR10 dataset
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar10)
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar10)

print(f"MNIST - Training: {len(mnist_train)} images, Test: {len(mnist_test)} images")
print(f"CIFAR10 - Training: {len(cifar10_train)} images, Test: {len(cifar10_test)} images")

Loading datasets...
MNIST - Training: 60000 images, Test: 10000 images
CIFAR10 - Training: 50000 images, Test: 10000 images


4. Data Loaders

In [23]:
# Data loaders
batch_size = 64
mnist_train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
mnist_test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)
cifar10_train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
cifar10_test_loader = DataLoader(cifar10_test, batch_size=batch_size, shuffle=False)


5. Feature Extractor Classes

In [24]:
# Feature extractor classes
class ResNet18Features(nn.Module):
    def __init__(self, original_model):
        super(ResNet18Features, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-2])  # Up to avgpool
        self.avgpool = original_model.avgpool

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x

class VGG16Features(nn.Module):
    def __init__(self, original_model):
        super(VGG16Features, self).__init__()
        self.features = original_model.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x

class GoogleNetFeatures(nn.Module):
    def __init__(self):
        super(GoogleNetFeatures, self).__init__()
        self.model = models.googlenet(pretrained=True)
        self.model.fc = nn.Identity()  # Replace fc with identity

    def forward(self, x):
        return self.model(x)


6. Parallel Model Architecture

In [25]:
# Parallel Model
class ParallelModel(nn.Module):
    def __init__(self, feature_extractor1, feature_extractor2, model_name1, model_name2, num_classes=10):
        super(ParallelModel, self).__init__()
        self.feature_extractor1 = feature_extractor1
        self.feature_extractor2 = feature_extractor2
        feature_sizes = {'resnet18': 512, 'vgg16': 512, 'googlenet': 1024}
        feature_size1 = feature_sizes[model_name1]
        feature_size2 = feature_sizes[model_name2]
        self.classifier = nn.Linear(feature_size1 + feature_size2, num_classes)

    def forward(self, x):
        features1 = self.feature_extractor1(x)
        features2 = self.feature_extractor2(x)
        combined_features = torch.cat((features1, features2), dim=1)
        output = self.classifier(combined_features)
        return output


7. Helper Functions

In [26]:
# Helper function to get feature extractors
def get_feature_extractor(model_name):
    if model_name == 'resnet18':
        model = models.resnet18(pretrained=True)
        return ResNet18Features(model).to(device)
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
        return VGG16Features(model).to(device)
    elif model_name == 'googlenet':
        return GoogleNetFeatures().to(device)

8. Training and Evaluation Functions

In [27]:
# Training and evaluation functions
def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    return accuracy

def compute_confusion_matrix(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    cm = confusion_matrix(all_targets, all_preds)
    return cm

def train_parallel_model(parallel_model, model_name1, model_name2, train_loader, test_loader, num_epochs, base_lr, classifier_lr):
    optimizer = optim.Adam([
        {'params': parallel_model.feature_extractor1.parameters(), 'lr': base_lr},
        {'params': parallel_model.feature_extractor2.parameters(), 'lr': base_lr},
        {'params': parallel_model.classifier.parameters(), 'lr': classifier_lr}
    ])
    criterion = nn.CrossEntropyLoss()

    print(f"Training for {num_epochs} epochs...")
    for epoch in range(num_epochs):
        loss, train_acc = train_one_epoch(parallel_model, train_loader, optimizer, criterion)
        print(f'Epoch {epoch+1}/{num_epochs}: Loss={loss:.4f}, Train Acc={train_acc:.2f}%')

    test_acc = evaluate(parallel_model, test_loader)
    print(f'Final Test Accuracy: {test_acc:.2f}%')

    cm = compute_confusion_matrix(parallel_model, test_loader)
    print(f'Confusion Matrix for Parallel {model_name1}+{model_name2}:\n{cm}')

    return test_acc

9. Main Execution Block

In [28]:
# For notebook environment - Now runs on both datasets
datasets = ['mnist', 'cifar10']
all_results = {}  # Dictionary to store results for all datasets

for dataset in datasets:
    # Select dataset
    if dataset == 'mnist':
        train_loader = mnist_train_loader
        test_loader = mnist_test_loader
        dataset_name = 'MNIST'
    else:
        train_loader = cifar10_train_loader
        test_loader = cifar10_test_loader
        dataset_name = 'CIFAR-10'

    print(f'\n=== Training parallel models on {dataset_name} ===')

    # Define model pairs to train
    pairs = [('resnet18', 'vgg16'), ('resnet18', 'googlenet'), ('vgg16', 'googlenet')]
    dataset_results = {}  # Results for this specific dataset

    # Train each model pair
    for model1_name, model2_name in pairs:
        print(f'\n\nTraining parallel model: {model1_name}+{model2_name}')
        feature_extractor1 = get_feature_extractor(model1_name)
        feature_extractor2 = get_feature_extractor(model2_name)
        parallel_model = ParallelModel(feature_extractor1, feature_extractor2, model1_name, model2_name).to(device)

        # Train model
        acc = train_parallel_model(
            parallel_model,
            model1_name,
            model2_name,
            train_loader,
            test_loader,
            num_epochs=10,  # Reduced from 10 for quicker execution
            base_lr=1e-4,
            classifier_lr=1e-3
        )

        # Store results for this dataset
        dataset_results[f"{model1_name}+{model2_name}"] = acc

    # Store results for this dataset in the overall results dictionary
    all_results[dataset_name] = dataset_results

    # Print summary of results for this dataset
    print(f"\n=== Results Summary for {dataset_name} ===")
    for model_pair, accuracy in dataset_results.items():
        print(f"{model_pair}: {accuracy:.2f}%")

# Print comparison across datasets
print("\n=== Comparison Across Datasets ===")
for model_pair in [f"{m1}+{m2}" for m1, m2 in pairs]:
    mnist_acc = all_results['MNIST'][model_pair]
    cifar_acc = all_results['CIFAR-10'][model_pair]
    print(f"{model_pair}: MNIST = {mnist_acc:.2f}%, CIFAR-10 = {cifar_acc:.2f}%")



=== Training parallel models on MNIST ===


Training parallel model: resnet18+vgg16
Training for 10 epochs...
Epoch 1/10: Loss=0.0568, Train Acc=98.31%
Epoch 2/10: Loss=0.0216, Train Acc=99.35%
Epoch 3/10: Loss=0.0172, Train Acc=99.47%
Epoch 4/10: Loss=0.0140, Train Acc=99.57%
Epoch 5/10: Loss=0.0106, Train Acc=99.68%
Epoch 6/10: Loss=0.0087, Train Acc=99.71%
Epoch 7/10: Loss=0.0118, Train Acc=99.64%
Epoch 8/10: Loss=0.0079, Train Acc=99.75%
Epoch 9/10: Loss=0.0055, Train Acc=99.83%
Epoch 10/10: Loss=0.0078, Train Acc=99.76%
Final Test Accuracy: 99.41%
Confusion Matrix for Parallel resnet18+vgg16:
[[ 979    0    0    0    0    0    1    0    0    0]
 [   5 1123    0    2    0    0    1    4    0    0]
 [   1    0 1028    0    0    0    0    3    0    0]
 [   0    0    1 1009    0    0    0    0    0    0]
 [   0    0    0    0  980    0    0    0    0    2]
 [   0    0    0   12    0  879    1    0    0    0]
 [   1    2    0    0    0    0  953    0    2    0]
 [   0    2    1    1  

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date we

Training for 10 epochs...
Epoch 1/10: Loss=0.0593, Train Acc=98.31%
Epoch 2/10: Loss=0.0224, Train Acc=99.33%
Epoch 3/10: Loss=0.0180, Train Acc=99.47%
Epoch 4/10: Loss=0.0147, Train Acc=99.56%
Epoch 5/10: Loss=0.0134, Train Acc=99.57%
Epoch 6/10: Loss=0.0136, Train Acc=99.59%
Epoch 7/10: Loss=0.0114, Train Acc=99.67%
Epoch 8/10: Loss=0.0108, Train Acc=99.70%
Epoch 9/10: Loss=0.0066, Train Acc=99.81%
Epoch 10/10: Loss=0.0093, Train Acc=99.75%
Final Test Accuracy: 99.55%
Confusion Matrix for Parallel resnet18+googlenet:
[[ 979    0    0    0    0    0    1    0    0    0]
 [   0 1133    1    0    0    0    0    1    0    0]
 [   1    0 1029    0    0    0    0    2    0    0]
 [   0    0    0 1008    0    1    0    0    1    0]
 [   0    0    0    0  972    0    0    0    0   10]
 [   0    0    0    3    0  886    0    0    1    2]
 [   4    1    0    0    0    1  946    0    5    1]
 [   0    3    2    0    1    0    0 1021    0    1]
 [   0    0    0    0    0    0    0    0  973    1

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.

Training for 10 epochs...
Epoch 1/10: Loss=0.0673, Train Acc=97.91%
Epoch 2/10: Loss=0.0219, Train Acc=99.33%
Epoch 3/10: Loss=0.0146, Train Acc=99.57%
Epoch 4/10: Loss=0.0143, Train Acc=99.53%
Epoch 5/10: Loss=0.0099, Train Acc=99.69%
Epoch 6/10: Loss=0.0120, Train Acc=99.65%
Epoch 7/10: Loss=0.0109, Train Acc=99.67%
Epoch 8/10: Loss=0.0072, Train Acc=99.79%
Epoch 9/10: Loss=0.0068, Train Acc=99.78%
Epoch 10/10: Loss=0.0081, Train Acc=99.75%
Final Test Accuracy: 99.51%
Confusion Matrix for Parallel vgg16+googlenet:
[[ 980    0    0    0    0    0    0    0    0    0]
 [   0 1135    0    0    0    0    0    0    0    0]
 [   0    1 1030    0    0    0    0    1    0    0]
 [   0    0    0 1005    0    5    0    0    0    0]
 [   0    0    0    0  975    0    0    0    0    7]
 [   0    0    0    1    0  891    0    0    0    0]
 [   4    3    0    0    1    6  943    0    1    0]
 [   0    4    4    0    0    0    0 1020    0    0]
 [   0    0    1    2    0    0    0    0  971    0]
 

...

Training for 10 epochs...
Epoch 1/10: Loss=0.3111, Train Acc=89.43%
Epoch 2/10: Loss=0.1026, Train Acc=96.42%
Epoch 3/10: Loss=0.0583, Train Acc=97.99%
Epoch 4/10: Loss=0.0463, Train Acc=98.46%
Epoch 5/10: Loss=0.0440, Train Acc=98.53%
Epoch 6/10: Loss=0.0335, Train Acc=98.87%
Epoch 7/10: Loss=0.0323, Train Acc=98.81%
Epoch 8/10: Loss=0.0347, Train Acc=98.85%
Epoch 9/10: Loss=0.0321, Train Acc=98.93%
Epoch 10/10: Loss=0.0240, Train Acc=99.18%
Final Test Accuracy: 93.98%
Confusion Matrix for Parallel resnet18+vgg16:
[[950   2   7   3   5   1   1   1  14  16]
 [  6 967   0   0   0   0   1   0   6  20]
 [ 15   0 928  12  10  10  21   2   2   0]
 [  7   1  14 873  21  47  19  12   2   4]
 [  4   0   6  13 951   6   7  12   0   1]
 [  1   0  11  74  12 879   8  15   0   0]
 [  5   1   4   9   1   2 975   2   1   0]
 [  5   0   4   4  16  25   1 943   0   2]
 [ 21   4   3   5   0   0   2   0 958   7]
 [  3  14   0   2   1   0   0   0   6 974]]


Training parallel model: resnet18+googlenet


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date we

Training for 10 epochs...
Epoch 1/10: Loss=0.2853, Train Acc=90.45%
Epoch 2/10: Loss=0.0910, Train Acc=96.97%
Epoch 3/10: Loss=0.0547, Train Acc=98.10%
Epoch 4/10: Loss=0.0463, Train Acc=98.40%
Epoch 5/10: Loss=0.0482, Train Acc=98.41%
Epoch 6/10: Loss=0.0428, Train Acc=98.58%
Epoch 7/10: Loss=0.0367, Train Acc=98.78%
Epoch 8/10: Loss=0.0311, Train Acc=98.95%
Epoch 9/10: Loss=0.0352, Train Acc=98.85%
Epoch 10/10: Loss=0.0295, Train Acc=99.08%
Final Test Accuracy: 94.94%
Confusion Matrix for Parallel resnet18+googlenet:
[[938   2  15   7   3   0   4   0  19  12]
 [  0 993   0   1   0   0   0   0   2   4]
 [  1   0 952  14  11  10   8   2   2   0]
 [  2   2   9 902   9  56  11   3   4   2]
 [  2   0   4  13 967   4   0  10   0   0]
 [  0   0   9  42  16 924   2   6   0   1]
 [  2   0  13  15   6   2 962   0   0   0]
 [  2   0   5   9  17   6   0 959   0   2]
 [ 17   7   4   1   0   0   1   0 950  20]
 [  2  44   1   2   0   0   0   0   4 947]]


Training parallel model: vgg16+googlenet


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.

Training for 10 epochs...
Epoch 1/10: Loss=0.3081, Train Acc=89.64%
Epoch 2/10: Loss=0.1006, Train Acc=96.61%
Epoch 3/10: Loss=0.0568, Train Acc=98.09%
Epoch 4/10: Loss=0.0425, Train Acc=98.49%
Epoch 5/10: Loss=0.0371, Train Acc=98.74%
Epoch 6/10: Loss=0.0364, Train Acc=98.72%
Epoch 7/10: Loss=0.0255, Train Acc=99.14%
Epoch 8/10: Loss=0.0296, Train Acc=99.01%
Epoch 9/10: Loss=0.0229, Train Acc=99.26%
Epoch 10/10: Loss=0.0234, Train Acc=99.22%
Final Test Accuracy: 94.39%
Confusion Matrix for Parallel vgg16+googlenet:
[[960   2  10   3   0   0   0   1  22   2]
 [  1 987   1   0   0   0   0   0   2   9]
 [  6   0 942   8  11  17   8   6   2   0]
 [  5   1  17 821  18 103  11  21   2   1]
 [  5   0  11   6 952   5   1  19   1   0]
 [  1   0   4  30  13 934   1  16   0   1]
 [  3   0  15   8   6   7 961   0   0   0]
 [  0   0   5   3  11   8   1 972   0   0]
 [ 14   3   4   3   0   1   1   1 968   5]
 [  7  40   0   1   0   1   0   1   8 942]]

=== Results Summary for CIFAR-10 ===
resnet18+